In [61]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import glob

In [75]:
def detectAndDescribe(image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        sift = cv2.xfeatures2d.SIFT_create()
        (kps, features) = sift.detectAndCompute(gray, None)
        
        #kps = np.float32([kp.pt for kp in kps])
        return (kps, features)

In [76]:
def ComputeMatches(a_images,b_images):

    for imageA, imageB in zip(a_images, b_images):
        #imageA = cv2.imread(img_a)
        #imageB = cv2.imread(img_b)

        imageA = cv2.resize(imageA,(400,200))
        imageB = cv2.resize(imageB,(400,200))

        (kpsA, descriptorA) = detectAndDescribe(imageA)
        (kpsB, descriptorB) = detectAndDescribe(imageB)

        # match features between the two images
        matcher = cv2.BFMatcher()
        rawMatches =matcher.knnMatch(descriptorA, descriptorB, 2)
        matches_backward = matcher.knnMatch(descriptorB, descriptorA, 2)


        matches = []
        ratio = 0.7
        
        #David Lowe's ratio test and cross-checking
        for m,n in rawMatches:
            if m.distance<(ratio*n.distance):
                for mm, nn in matches_backward:
                    if mm.trainIdx == m.queryIdx and mm.queryIdx == m.trainIdx:
                        matches.append(m)
                        break
        matchesMask=None
        #apply the ransac mask to the matches    
        if len(matches) >=4:
            ptsA = np.float32([kpsA[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
            ptsB = np.float32([kpsB[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
            
            (H, mask) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,20)
            
            matchesMask = mask.ravel().tolist()
            print("matchesMask: ", len(matchesMask))
        
       
    
        #3- Threshold based on distance. (The distance must not be constant and should be derived from an analysis of the data distribution)
        # if len(matches) > 0:
        #     distances = [m.distance for m in matches]
            
        #     median_distance = np.median(distances)
        #     threshold = median_distance
        #     matches = [m for m in matches if m.distance < threshold]    

        #4- Draw the matches on the images and display the images


        similarity_score = len(matches) / min(len(kpsA), len(kpsB))
    
        if similarity_score > 0.2:
            print("The images are similar")
            
        else:
            print("The images are not similar")
        print("similarity score: ", similarity_score)
        #result = cv2.drawMatches(imageA, kpsA, imageB, kpsB, matches, None)
        #cv2.line(result, (0, 200), (400, 200), (0, 0, 255), 2)
        #cv2.imshow("result", result)
        print("matches: ", len(matches))
        if matchesMask!=None and len(matches)==len(matchesMask)  :   
            result=cv2.drawMatches(imageA,kpsA,imageB,kpsB,matches,None,
                            matchesMask=matchesMask,# draw only inliers
                            flags=2)
            cv2.imshow("result", result)

        cv2.imshow("imageA",imageA)
        cv2.imshow("imageB",imageB)
        cv2.waitKey(0)




In [85]:
import numpy as np
images = glob.glob("C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data/*")
print(images)
a_images = [img for img in images if img.endswith("a.jpeg")]
b_images = [img for img in images if img.endswith("b.jpeg")]
c_images = [img for img in images if img.endswith("c.png")]

a_images.sort()
b_images.sort()
c_images.sort()
max_length = max(len(a_images), len(b_images), len(c_images))
for i in range(max_length):
    if i < len(a_images):
        img_a = a_images[i]
        imageA = cv2.imread(img_a,cv2.IMREAD_UNCHANGED)
   

    if i < len(b_images):
        img_b = b_images[i]
        imageB = cv2.imread(img_b,cv2.IMREAD_UNCHANGED)
    

    if i < len(c_images):
        img_c = c_images[i]
        imageC = cv2.imread(img_c,cv2.IMREAD_UNCHANGED)
    

# Process images

    image_number_a = int(''.join(filter(str.isdigit, img_a)))
    image_number_b = int(''.join(filter(str.isdigit, img_b)))
    image_number_c = int(''.join(filter(str.isdigit, img_c)))
    if image_number_a == image_number_b == image_number_c:

            ComputeMatches(imageA,imageB)
            ComputeMatches(imageA,imageC)
            ComputeMatches(imageB,imageC)
    else:
        ComputeMatches(imageA,imageB)
    

['C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image1a.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image1b.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image2a.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image2b.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image3a.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image3b.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image4a.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image4b.jpeg', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/assignment data\\image4c.png', 'C:/Users/marwa/Downloads/CV Labs/assignment data-20240329T203755Z-001/as

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x16764f4e::Set<1,-1,-1>,struct cv::impl::A0x16764f4e::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1
